Read Data

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('./weatherAUS.csv')
print('Size of weather data frame is :',df.shape)
#Let us see how our data looks like!
df[0:5]

Size of weather data frame is : (145460, 24)


Date Location  MinTemp  ...  RainToday  RISK_MM  RainTomorrow
0  2008-12-01   Albury     13.4  ...         No      0.0            No
1  2008-12-02   Albury      7.4  ...         No      0.0            No
2  2008-12-03   Albury     12.9  ...         No      0.0            No
3  2008-12-04   Albury      9.2  ...         No      1.0            No
4  2008-12-05   Albury     17.5  ...         No      0.2            No

[5 rows x 24 columns]

In [2]:
df.shape

(145460, 24)

Drop RISK_MM column

In [0]:
df.drop(columns=['RISK_MM'], axis = 1, inplace = True)

In [4]:
df.shape

(145460, 23)

# Data Pruning
### 1. Deal with Nan & null values
Count null values

In [5]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

Few columns such as Sunshine, Evaporation, Cloud9am, Cloud3pm have greater than 50000 null values which is almost 33% or more. We can either **ignore** these columns or **fill** the NaN values with some dummy values. Here, we will these drop the features.  
 
Below, we are using ExtraTreesClassifier to derive feature importance of the above four features, for comparison we take few other features into consideration

In [6]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [7]:
"""
columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation']

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
print(df.shape)
model = ExtraTreesClassifier()
df = df.dropna(axis=0, subset=['RainTomorrow'])
print(df.shape)
X = df[columns].fillna(df[columns].mean(), axis=1)
y = df[['RainTomorrow']]
print(X.shape)
model.fit(X, y)

print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()
"""

"\ncolumns=['Sunshine','Evaporation','Cloud3pm','Cloud9am', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation']\n\nfrom sklearn.ensemble import ExtraTreesClassifier\nimport matplotlib.pyplot as plt\nprint(df.shape)\nmodel = ExtraTreesClassifier()\ndf = df.dropna(axis=0, subset=['RainTomorrow'])\nprint(df.shape)\nX = df[columns].fillna(df[columns].mean(), axis=1)\ny = df[['RainTomorrow']]\nprint(X.shape)\nmodel.fit(X, y)\n\nprint(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers\n#plot graph of feature importances for better visualization\n\nfeat_importances = pd.Series(model.feature_importances_, index=X.columns)\nfeat_importances.nlargest(10).plot(kind='barh')\nplt.show()\n"

Clearly we can drop sunshine and evaporation are two important features, but the fact to notice is that these metrics are obtained after dropping NA values and looking at the shape it is clear that we have lost 2/3s of the data. But we could still remove cloud features and try.

Again there is heavy loss of data. Hence, it would be prudent to drop these features.

In [8]:
## Drop Null valued columns
df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am'], axis=1, inplace = True)
print(df.columns.values)

['Date' 'Location' 'MinTemp' 'MaxTemp' 'Rainfall' 'WindGustDir'
 'WindGustSpeed' 'WindDir9am' 'WindDir3pm' 'WindSpeed9am' 'WindSpeed3pm'
 'Humidity9am' 'Humidity3pm' 'Pressure9am' 'Pressure3pm' 'Temp9am'
 'Temp3pm' 'RainToday' 'RainTomorrow']


Other columns do have null values in them, we could either ignore the rows or substitute them mean values of the column. We can try both approaches, first dropping the rows.

1. Some columns as RainToday, Rain Tomorrow are non-numeric columns, while substituting the NaN values by mean, we can first change the values to 0 and 1. 

*Dropping rows with Rain Tomorrow = NA values as these rows not useful intraining or testing.*

In [0]:
df.dropna(axis=0, subset=['RainTomorrow'], inplace=True)

### 1.1 Handling binary data

In [0]:
df['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
df['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

### 1.2 Handling Nan/NA values
Now susbtituing values

1. Nan Numeric values are changed to column means
2. NA categorical values are changed to mode of the column

In [0]:
# Prefill missing values with mean
df.fillna(df.mean(), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)
# Drop rows with null values
# df.dropna(how='any', inplace=True)
# df.shape

Dropping null valued rows (null in any column) resulted in 2/3 loss of data.

#### Should we do Z-score to remove outliers?

Currently, we will keep the outliers, we can also check with removing the outliers.

### 2. Handle Date value
We must change date variable and split it to month, year, date. It would be interesting to check if month, year & day possibly have any correlation with the result.

In [0]:
df['Date'] = pd.to_datetime(df['Date'])

# Extract date components
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Drop date column
df.drop('Date', axis=1, inplace = True)

### 3. Handling categorical data

There are number of columns which have categorical data for eg:- WindGustDir, Location. This data needs to be represented such as it is palapable to the models. Hence, we encode these data columns as one hot vector, by adding an additional columns. We will not be using Label Encoder as experimenting with it showed that it decreases the f-1 score. 
// TODO Localtion effect

In [13]:
categorical_columns = ['WindGustDir', 'WindDir3pm', 'WindDir9am', 'Location']
#df = pd.get_dummies(df, columns=categorical_columns) #, dummy_na=True
#df[0:5]
# TODO: Check categorical data for unique values
from sklearn import preprocessing 

#df.fillna(df.mode().iloc[0], inplace=True)
df= df.apply(preprocessing.LabelEncoder().fit_transform)
df[0:5]
#df = df_le #df_ohe

Location  MinTemp  MaxTemp  Rainfall  ...  RainTomorrow  Year  Month  Day
0         2      214      262         6  ...             0     1     11    0
1         2      153      285         0  ...             0     1     11    1
2         2      209      291         0  ...             0     1     11    2
3         2      171      314         0  ...             0     1     11    3
4         2      255      357        10  ...             0     1     11    4

[5 rows x 21 columns]

# Feature Scaling
Feature domination for eg:- Pressure might dominate temperature from above sample.
There are few scaling methods that can scale data reduce feature domination. 

1. Standard Scalar: scales features with mean 0.
2. MinMax Scalar: scales to 0 to 1 range.
3. QuantileTransformer: Fits data onto a normal distribution and reduces the effects of outliers

Before, we proceed, it would be a good excercise to get some statistical analysis on features.

In [14]:
df.describe()

Location        MinTemp  ...          Month            Day
count  142193.000000  142193.000000  ...  142193.000000  142193.000000
mean       23.740655     201.352992  ...       5.402544      14.715084
std        14.237503      64.295257  ...       3.426506       8.798150
min         0.000000       0.000000  ...       0.000000       0.000000
25%        11.000000     155.000000  ...       2.000000       7.000000
50%        24.000000     199.000000  ...       5.000000      15.000000
75%        36.000000     248.000000  ...       8.000000      22.000000
max        48.000000     389.000000  ...      11.000000      30.000000

[8 rows x 21 columns]

#### Inference :
It is clear from the std deviation that most of the data points are well surrounded around the mean. Scaling techniques like MinMacScalar might accidently include the influence of outliers. Nonetheless, we experiment with it.

Currently, as data contains one hot encoding it would be great to use MinMaxScalar. But we could essentially apply different transformation on categorical versus numerical data.

For simplicity, we first experiment with MinMaxScalar.

In [0]:
from sklearn import preprocessing

#scaler = preprocessing.StandardScaler()  # standard sclaer will scale some properties of one hot encoded data as well
scaler = preprocessing.QuantileTransformer() #(ignore_implicit_zeros=True) if used on one-hot encoded
#scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

### Scaled Data

In [16]:
df.iloc[0:10]

Location   MinTemp   MaxTemp  ...      Year  Month       Day
0  0.054054  0.580581  0.511512  ...  0.008509    1.0  0.000000
1  0.054054  0.238238  0.620621  ...  0.008509    1.0  0.050050
2  0.054054  0.552052  0.647147  ...  0.008509    1.0  0.083083
3  0.054054  0.336336  0.742242  ...  0.008509    1.0  0.116116
4  0.054054  0.780280  0.883884  ...  0.008509    1.0  0.149149
5  0.054054  0.644645  0.803303  ...  0.008509    1.0  0.181682
6  0.054054  0.628629  0.615616  ...  0.008509    1.0  0.214214
7  0.054054  0.254254  0.690190  ...  0.008509    1.0  0.247247
8  0.054054  0.365365  0.871872  ...  0.008509    1.0  0.280781
9  0.054054  0.563564  0.816817  ...  0.008509    1.0  0.313814

[10 rows x 21 columns]

# Feature Selection
There are multitude of methods for feature selection.
1. Univariate feature selection
2. ExtraTressClassifier to select features

In [17]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
X = df.loc[:,df.columns!='RainTomorrow']
y = scaler.inverse_transform(df[['RainTomorrow']])
selector = SelectKBest(mutual_info_classif, k=7)
selector.fit(X, y)
X_fs = selector.transform(X)
print(X.columns[selector.get_support(indices=True)]) #top 3 columns

ValueError: ignored

In [0]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

model = ExtraTreesClassifier()
y = y[['RainTomorrow']]
X = df.drop(labels=['RainTomorrow'], axis=1)
print(X.shape)
X_etc = model.fit(X, y)

print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
print(feat_importances.nlargest(10).keys())
plt.show()
X_etc = df[feat_importances.nlargest(10).keys().tolist()]

# Split into Train / Test data

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_etc,y,test_size=0.25)

## Training
1. Select models and apply hyper parameters.
2. Use ross validation (k-fold).
//explain
3. Use GridSearch CV
//explain

## Logistic Regression
### Train
About Hyper-Parameters
1. Trying both l1 & l2 regularization penalty
2. Balanced weights as classes are unbalanced

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
#clf_logreg = LogisticRegression(random_state=0, class_weight='balanced', C=100)
#clf_logreg.fit(X_train,y_train)

## Hyper parameters

hyperparameters = dict(
    C=np.logspace( 2, 4, 6),     # regularization penalty 
    penalty=['l1', 'l2'],        # regularization hyperparameter
    #tol=np.logspace( 2, 4),
    #solver=['liblinear', 'saga'],# solvers to use
    class_weight=['balanced'],
    n_jobs=-1)                   # use all cpus 

hyperparameters=dict()           # dry run

clf = GridSearchCV(LogisticRegression(), hyperparameters, cv=3, verbose=0)

#### Train and print best model

In [0]:
best_model = clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(best_model.best_params_)

### Test & F1 score, Accuracy

In [0]:
from sklearn.metrics import classification_report
y_pred = best_model.predict(X_test)
report = classification_report(y_test,y_pred, labels=[0.0, 1.0])
print('Report :\n',report)

## Decision Trees
### Train
About Hyper-Parameters


In [0]:
#Decision Tree Classifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

## Hyper parameters
hyperparameters = dict(
    loss=['deviance', 'exponential'],
    learning_rate=[0.1, 0.0001, 1],     # regularization penalty 
    min_samples_split=[1, 2, 4],        # regularization hyperparameter
    #tol=np.logspace( 2, 4),
    subsample=[0.8, 1, 1.2],# solvers to use
    max_depth=[3,5,7],
    min_samples_leaf=[1,3,7])                   # use all cpus 

hyperparameters=dict()           # dry run

model = Pipeline([
        ('sampling', SMOTE()),  ## Dry run
        ('classification', GradientBoostingClassifier())
    ])

clf = GridSearchCV(model, hyperparameters, cv=3, verbose=0)

### Train and print best model

In [0]:
best_model = clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(best_model.best_params_)

### Test & F1 score, Accuracy

In [0]:
from sklearn.metrics import classification_report
y_pred = best_model.predict(X_test)
score = classification_report(y_test,y_pred, labels=[0.0, 1.0])
print('Accuracy :',score)

**Thus, GradientBoostedTrees have a higher recall for class 0 ( not raining ) and lower f1score for label 1.0 than LogisticReegrssion.**